# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хранить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [2]:
#with open('data/2ch_corpus.txt', 'r', encoding='utf-8') as file1:
#    dvach = file1.read()

with open('data/lenta.txt', 'r', encoding='utf-8') as file2:
    news = file2.read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [32]:
#sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:50000])]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [217]:
len(sentences_news)

32250

In [218]:
sentences_news_check = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[5000001:5005000])]

In [220]:
len(sentences_news_check)

40

In [33]:
#unigrams_dvach = Counter()
#bigrams_dvach = Counter()
#trigrams_dvach = Counter()

#for sentence in sentences_dvach:
#    unigrams_dvach.update(sentence)
#    bigrams_dvach.update(ngrammer(sentence))
#    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [204]:
matrix_news = lil_matrix((len(bigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [210]:
def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        row_probabilities = matrix[current_idx].toarray()[0]

        # Проверка на NaN или нулевые вероятности
        if np.any(np.isnan(row_probabilities)) or row_probabilities.sum() == 0:
            chosen = np.random.choice(matrix.shape[1])
        else:
            # Нормализация вероятностей
            row_probabilities /= row_probabilities.sum()
            chosen = np.random.choice(matrix.shape[1], p=row_probabilities)

        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            chosen = bigram2id['<start> <start>']

        words = id2bigram_news[current_idx]
        word1, word2 = words.split()
        word3 = id2word[chosen]

        new_bigram = word2 + ' ' + word3

        if new_bigram in bigram2id:
            current_idx = bigram2id[new_bigram]
        else:
            new_bigram = '<start> ' + word3
            current_idx = bigram2id[new_bigram]

    return ' '.join(text)


In [211]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

другой опрос но и в перспективе так как второй транш кредита в 35 из них 2438 дети до 16 миллионов тройских унций около 500 семей москвичей остались без связи из-за аварии восстановлена по данным российского оборонного ведомства в шали около 150 миллионов рублей 
 окончательную точку здесь поставит суд добавил владимир козлов 
 переговоры скаутов c новыми владельцами домена результатов пока не нашло доказательств существования подобных тайников но свидетельства похожих акций советской разведки были найдены пути решения вопросов связанных с высокими технологиями а также итоги 1999 года на указанной частоте ведется круглосуточная трансляция новой радиопрограммы объявляемой в эфире радиостанции эхо


In [203]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

газпром является владельцем легкового автомобиля газ-3111 волга 
 член мгик требует через суд или суды за рубежом кправительству и минфину рф полномочия по регистрации выпусков облигаций цб а также новый генеральный секретарь совета безопасности марат тажин распорядился создать новое государство от участия в выборах в госдуму россии о восстановлении регистрации списка федерального избирательного объединения снимет свою кандидатуру на президентских выборах сейчас имеет визу h-1 b и тех кто заказал дело лазаренко кто довел народ украины до сих пор находится под стражей 
 управделами считает что представителя президента рф правительстварф каждой из которых на одного человека· мяса 31,5 кг · молока


In [300]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

платить за акции по завышенным ценам 
 полиция началарасследование но никакие подробности пока не установлены 
 крушение произошло недалеко от стамбула 
 напомним что в городе строительство парламентского центра чтобы перевести в федеральное подчинение 
 пожару была присвоена степень почетного доктора 
 подсчет голосов еще продолжается в субботу в дорожную аварию он отделалася лишь несколькими царапинами на голове и проникнув в здание генпрокуратуры и нормальные условия хранения создать в россии альберта давлеева главная цель этого этапа окончательное уничтожение бандформирований на северном кавказе приступает ко второму чтению крупнейший британский банк barclays принял свое решение заключением британских врачей 
 также велось прочесывание


----------------------------------------

In [224]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [239]:
text = ''

for sentence in sentences_news_check:
    for word in sentence:
        word += ' '
        text += word


In [256]:
text1 = normalize(text)

In [261]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts, trigram_counts):
    prob = 0
    for ngram in ngrammer(text, n=3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(text1)

In [301]:
perplexity(*compute_joint_proba_markov_assumption(text1, unigrams_news, bigrams_news, trigrams_news))

15213.303465450957

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [263]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [294]:
def generate_with_beam_search(matrix, id2word, bigram2id, n=100, max_beams=5, start='<start> <start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            a = beam.sequence[-2:]
            aa = ' '.join(a)
            if len(aa) == 1:
                aa = '<start> ' + aa
            if aa not in bigram2id:
                aa = '<start> <start>'

            last_id = bigram2id[aa]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence

    
    return ' '.join(best_sequence)

In [281]:
print(generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, start='другой'))

другой чеченский боевик шамиль басаев руслан гелаев ваха арсанов ахмед закаев и другие документы готовятся дляотправки в ближайшее времяможет начаться спецоперация против бандформирований которые терроризируют мирное население и работу предприятий перерабатывающей промышленности санкт-петербурга составил 234 процента по сравнению с прошлым годом когда финансы были расшатаны долги росли и дефицит бюджета достиг 8 процентов от суммы привлеченных валютных депозитов и 7 представителей glaxo и smithkline beecham планируют провести диверсии и теракты не только его возраст но и в ближайшее время может быть обжаловано в кассационной коллегии верховного суда россии отклонила жалобу избирательного объединения или блока или создать инициативную группу из 147 человек


In [295]:
print(generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, start='куда'))

куда по его словам на сегодняшний день уже четверо членов ндр могут занять в нижней части бюллетеня расположено поле для гольфа оставив за собой право предъявить российскойстороне финансовые претензии по возможному урегулированию конфликта на лфз заключается в том числе и распространение вредоносных программ tribe flood network tfn и схожая с ней поставками товаров на которые содержится московское правительство работает не покладая рук но реально завершить свою работу после обеденного перерыва участниками митинга были избиты заведующая общим отделом аппарата правительства путин будет исполнять олег казаков ранее занимавший должность главного координатора ес по внешней политике нового правительства индии новая ядерная доктрина страны <end>


In [296]:
print(generate_with_beam_search(matrix_news, id2word_news, bigram2id_news))

<start> <start> по в соответствии с указом президента посмертно награждены медалью анания ширакаци а депутаты ряда оппозиционных организаций вообще отказались от тактики позиционных боев и используют ее преимущественно для чатов игр переписки или поиска музыкальных файлов из интернета журналистами труда комсомольской правды новой газеты вячеслав измайлов яблоко автор и ведущей телепрограммы момент истины андрей караулов выдвинут избирателями глава общероссийского коммунистического общественного политического движения андрей брежнев самовыдвижение знаменитая лыжница елена вяльбе отизбирательного объединения кедр депутат госдумы второго созыва последовательно отстаивала курс на афганистан и направится в давос уже объявили 54 сообщает интерфакс телерадиокомпания петербург возобновляет свои передачи с 18.00 в пятницу в столице


In [297]:
print(generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, start='голубое небо'))

голубое небо по в соответствии с указом президента посмертно награждены медалью анания ширакаци а депутаты ряда оппозиционных организаций вообще отказались от тактики позиционных боев и используют ее преимущественно для чатов игр переписки или поиска музыкальных файлов из интернета журналистами труда комсомольской правды новой газеты вячеслав измайлов яблоко автор и ведущей телепрограммы момент истины андрей караулов выдвинут избирателями глава общероссийского коммунистического общественного политического движения андрей брежнев самовыдвижение знаменитая лыжница елена вяльбе отизбирательного объединения кедр депутат госдумы второго созыва последовательно отстаивала курс на афганистан и направится в давос уже объявили 54 сообщает интерфакс телерадиокомпания петербург возобновляет свои передачи с 18.00 в пятницу в столице


In [299]:
print(generate_with_beam_search(matrix_news, id2word_news, bigram2id_news, start='совет федерации'))

совет федерации может в ближайшеевремя вообще утратить свои функции такой прогноз якушкина сильным сомнениям подверг эту информацию подтвердил министр информации югославии горан матич в интервью риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в штабе объединенной группировки войск на северном кавказе генерал-полковник виктор казанцев геннадий трошев владимиршаманов и валентин корабельников а также по факсу по электронной почте или факсу и согласно идее автора должны вывешиваться в местах лишения свободы с отбыванием наказания в колонии усиленного режима приговорил суд армянского города гюмри двух российских транспортных самолетов ввс австралии для эвакуации персонала оон из города то братьям меньшим надо подыскать
